 #  <div style="text-align: center">Probability of Earthquake  </div> 
###  <div style="text-align: center">CLEAR DATA. MADE MODEL. </div> 
<img src='http://s8.picofile.com/file/8355280718/pro.png' width=400 height=400>
<div style="text-align:center"> last update: <b>17/03/2019</b></div>



You can Fork code  and  Follow me on:

> ###### [ GitHub](https://github.com/mjbahmani/10-steps-to-become-a-data-scientist)
> ###### [Kaggle](https://www.kaggle.com/mjbahmani/)
-------------------------------------------------------------------------------------------------------------
 <b> I hope you find this kernel helpful and some <font color='red'>UPVOTES</font> would be very much appreciated.</b>
    
 -----------

 <a id="top"></a> <br>
## Notebook  Content
1. [Introduction](#1)
1. [Load packages](#2)
    1. [import](21)
    1. [Setup](22)
    1. [Version](23)
1. [Problem Definition](#3)
    1. [Problem Feature](#31)
    1. [Aim](#32)
    1. [Variables](#33)
    1. [Evaluation](#34)
1. [Exploratory Data Analysis(EDA)](#4)
    1. [Data Collection](#41)
    1. [Visualization](#42)
    1. [Data Preprocessing](#43)
1. [Model Development](#5)
    1. [SVM](#5)
    1. [LGBM](#5)
1. [submission](#6)
1. [References](#7)


 <a id="1"></a> <br>
## 1- Introduction
**Forecasting earthquakes** is one of the most important problems in Earth science. If you agree, the earthquake forecast is likely to be related to the concepts of **probability**. In this kernel, I try to look at the prediction of the earthquake with the **help** of the concepts of **probability** .
<img src='https://www.preventionweb.net/files/52472_largeImage.jpg' width=600 height=600 >
For anyone taking first steps in data science, **Probability** is a must know concept. Concepts of probability theory are the backbone of many important concepts in data science like inferential statistics to Bayesian networks. It would not be wrong to say that the journey of mastering statistics begins with **probability**.

Before starting, I have to point out that I used the following great kernel:
[https://www.kaggle.com/inversion/basic-feature-benchmark](https://www.kaggle.com/inversion/basic-feature-benchmark)

 <a id="2"></a> <br>
 ## 2- Load packages
  <a id="21"></a> <br>
## 2-1 Import

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor,Pool
import matplotlib.patches as patch
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import norm
from scipy import linalg
from sklearn import svm
import lightgbm as lgb
import xgboost as xgb
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import random
import time
import glob
import sys
import os

 <a id="22"></a> <br>
##  2-2 Setup

In [ ]:
%matplotlib inline
%precision 4
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
pd.set_option("display.precision", 15)

 <a id="23"></a> <br>
## 2-3 Version


In [ ]:
print('pandas: {}'.format(pd.__version__))
print('numpy: {}'.format(np.__version__))
print('Python: {}'.format(sys.version))

<a id="3"></a> 
<br>
## 3- Problem Definition
I think one of the important things when you start a new machine learning project is Defining your problem. that means you should understand business problem.( **Problem Formalization**)

Problem Definition has four steps that have illustrated in the picture below:
<img src="http://s8.picofile.com/file/8344103134/Problem_Definition2.png" width=400 height=400>

> <font color='red'>Note</font> : **Current scientific studies related to earthquake forecasting focus on three key points:** 
1. when the event will occur
1. where it will occur
1. how large it will be.


<a id="31"></a> 
### 3-1 Problem Feature

1.     Train.csv - A single, continuous training segment of experimental data.
1.     Test - A folder containing many small segments of test data.
1.     Slample_sumbission.csv - A sample submission file in the correct format.


<a id="32"></a> 
### 3-2 Aim
In this competition, you will address <font color='red'><b>WHEN</b></font> the earthquake will take place.

<a id="33"></a> 
### 3-3 Variables

1.     acoustic_data - the seismic signal [int16]
1.     time_to_failure - the time (in seconds) until the next laboratory earthquake [float64]
1.     seg_id - the test segment ids for which predictions should be made (one prediction per segment)


<a id="34"></a> 
## 3-4 evaluation
Submissions are evaluated using the [mean absolute error](https://en.wikipedia.org/wiki/Mean_absolute_error) between the predicted time remaining before the next lab earthquake and the act remaining time.
<img src='https://wikimedia.org/api/rest_v1/media/math/render/svg/3ef87b78a9af65e308cf4aa9acf6f203efbdeded'>

<a id="4"></a> 
## 4- Exploratory Data Analysis(EDA)
 In this section, we'll analysis how to use graphical and numerical techniques to begin uncovering the structure of your data. 
 
* Which variables suggest interesting relationships?
* Which observations are unusual?
* Analysis of the features!
By the end of the section, you'll be able to answer these questions and more, while generating graphics that are both insightful and beautiful.  then We will review analytical and statistical operations:

*  Data Collection
*  Visualization
*  Data Preprocessing
*  Data Cleaning

 <a id="41"></a> <br>
## 4-1 Data Collection

In [ ]:
print(os.listdir("../input/"))

In [ ]:
%%time
train = pd.read_csv('../input/train.csv' , dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

In [ ]:
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
submission.head()

### memory usage: 3.5 GB

In [ ]:
train.info()

In [ ]:
#train.describe()

In [ ]:
print("Train: rows:{} columns:{}".format(train.shape[0], train.shape[1]))

### Wow! so large(rows:629145480 columns:2) for playing with it, let's select just 150000 rows!

In [ ]:
train.head()

### There are 2624 files in test.zip.

In [ ]:
len(os.listdir(os.path.join('../input/', 'test')))

Also we have 2624  row same as number of test files, so this clear that we should predict time_to_failure for all of test files.

In [ ]:
submission.shape

 <a id="42"></a> <br>
## 4-2 Visualization
Because the size of the database is very large, for the visualization section, we only need to select a small subset of the data.

In [ ]:
# we change the size of Dataset due to play with it (just loaded %0001)
mini_train= pd.read_csv("../input/train.csv",nrows=150000)

In [ ]:
mini_train.describe()

In [ ]:
mini_train.shape

In [ ]:
mini_train.isna().sum()

In [ ]:
type(mini_train)

<a id="421"></a> 
### 4-2-1 hist

In [ ]:
#acoustic_data means signal
mini_train["acoustic_data"].hist();

<a id="425"></a> 
### 4-2-5 Time to failure histogram

In [ ]:
plt.plot(mini_train["time_to_failure"], mini_train["acoustic_data"])
plt.title("time_to_failure histogram")

<a id="426"></a> 
### 4-2-6 Distplot

In [ ]:
sns.distplot(mini_train["acoustic_data"])

<a id="427"></a> 
### 4-2-7 kdeplot

In [ ]:
sns.kdeplot(mini_train["acoustic_data"] )

 <a id="43"></a> <br>
## 4-3 Data Preprocessing


Because we have only one feature(**acoustic_data**), and the size of the training set is very large( more that 60000000 rows), it is a good idea to reduce the size of the training set with making new segment  and also to increase the number of attributes by using statistical attributes.

In [ ]:
# based on : https://www.kaggle.com/inversion/basic-feature-benchmark
rows = 150_000
segments = int(np.floor(train.shape[0] / rows))
segments

In [ ]:
X_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['ave', 'std', 'max', 'min'])
y_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])

### y_train is our target for prediction

In [ ]:
y_train.head()

### our train set with 4 new feature

In [ ]:
X_train.head()

> <font color='red'>Note:</font>  
**tqdm** means "progress" in Arabic (taqadum, تقدّم) and is an abbreviation for "I love you so much" in Spanish (te quiero demasiado). Instantly make your loops show a smart progress meter - just wrap any iterable with tqdm(iterable), and you're done! [https://tqdm.github.io/](https://tqdm.github.io/)

In [ ]:
for segment in tqdm(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = seg['acoustic_data'].values
    y = seg['time_to_failure'].values[-1]
    
    y_train.loc[segment, 'time_to_failure'] = y
    
    X_train.loc[segment, 'ave'] = x.mean()
    X_train.loc[segment, 'std'] = x.std()
    X_train.loc[segment, 'max'] = x.max()
    X_train.loc[segment, 'min'] = x.min()
    X_train.loc[segment, 'min'] = x.sum()
    

In [ ]:
X_train.head()

In [ ]:
X_train.info()

### Cheking missing Data

In [ ]:
def check_missing_data(df):
    flag=df.isna().sum().any()
    if flag==True:
        total = df.isnull().sum()
        percent = (df.isnull().sum())/(df.isnull().count()*100)
        output = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
        data_type = []
        # written by MJ Bahmani
        for col in df.columns:
            dtype = str(df[col].dtype)
            data_type.append(dtype)
        output['Types'] = data_type
        return(np.transpose(output))
    else:
        return(False)

In [ ]:
check_missing_data(train)

### Now we must create our X_test for submission

In [ ]:
X_test = pd.DataFrame(columns=X_train.columns, dtype=np.float64, index=submission.index)

In [ ]:
X_test.head()

In [ ]:
%%time
for seg_id in X_test.index:
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    
    x = seg['acoustic_data'].values
    
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()
    X_test.loc[seg_id, 'min'] = x.sum()

In [ ]:
X_test.shape

Now we have all of the data frames for applying ML algorithms. just adding some feature scaling.

In [ ]:
X=X_train.copy()
y=y_train.copy()

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)

In [ ]:
X_test_scaled = scaler.transform(X_test)

<a id="5"></a> <br>
# 5- Model Development

<a id="51"></a> <br>
# 5-1 SVM

In [ ]:
svm = NuSVR()
svm.fit(X_train_scaled, y_train.values.flatten())
y_pred_svm = svm.predict(X_train_scaled)

In [ ]:
score = mean_absolute_error(y_train.values.flatten(), y_pred_svm)
print(f'Score: {score:0.3f}')

<a id="52"></a> <br>
# 5-2 LGBM

In [ ]:
folds = KFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
params = {'objective' : "huber", 
               'boost':"gbdt",
               'metric':"mae",
               'boost_from_average':"false",
               'num_threads':8,
               'learning_rate' : 0.01,
               'num_leaves' : 54,
               'max_depth':-1,
               'tree_learner' : "serial",
               'feature_fraction' : 0.05,
               'bagging_freq' : 5,
               'bagging_fraction' : 0.4,
               'min_data_in_leaf' : 80,
               'min_sum_hessian_in_leaf' : 10.0,
               'verbosity' : 1}

In [ ]:
for fold_n, (train_index, valid_index) in enumerate(folds.split(X)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
        
    model = lgb.LGBMRegressor(**params, n_estimators = 50000, n_jobs = -1)
    model.fit(X_train, y_train, 
                    eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric='mae',
                    verbose=10000, early_stopping_rounds=200)
            
    y_pred_valid = model.predict(X_valid)
    y_pred_lgb = model.predict(X_test, num_iteration=model.best_iteration_)

# Catboost 

In [ ]:
train_pool = Pool(X,y)
cat_model = CatBoostRegressor(
                               iterations=3000,# change 25 to 3000 to get best performance 
                               learning_rate=0.03,
                               eval_metric='MAE',
                              )
cat_model.fit(X,y,silent=True)
y_pred_cat = cat_model.predict(X_test)

<a id="6"></a> <br>
# 6- submission

### submission for svm

In [ ]:
y_pred_svm= svm.predict(X_test_scaled)

In [ ]:
submission['time_to_failure'] = y_pred_cat
submission.to_csv('submission_svm.csv')

### Submission for LGBM

In [ ]:
submission['time_to_failure'] = y_pred_lgb
submission.to_csv('submission_lgb.csv')

### Submission for Catboost

In [ ]:
submission['time_to_failure'] = y_pred_cat
submission.to_csv('submission_cat.csv')

<a id="61"></a> <br>
# 6-1 blending

In [ ]:
blending = y_pred_svm*0.5 + y_pred_lgb*0.5 
submission['time_to_failure']=blending
submission.to_csv('submission_lgb_svm.csv')

you can follow me on:
> ###### [ GitHub](https://github.com/mjbahmani/)
> ###### [Kaggle](https://www.kaggle.com/mjbahmani/)

 <b>I hope you find this kernel helpful and some <font color='red'>UPVOTES</font> would be very much appreciated.<b/>
 

<a id="7"></a> <br>
# 7-References
1. [Basic Probability Data Science with examples](https://www.analyticsvidhya.com/blog/2017/02/basic-probability-data-science-with-examples/)
1. [How to self learn statistics of data science](https://medium.com/ml-research-lab/how-to-self-learn-statistics-of-data-science-c05db1f7cfc3)
1. [Probability statistics for data science- series](https://towardsdatascience.com/probability-statistics-for-data-science-series-83b94353ca48)
1. [basic-statistics-in-python-probability](https://www.dataquest.io/blog/basic-statistics-in-python-probability/)
1. [tutorialspoint](https://www.tutorialspoint.com/python/python_poisson_distribution.htm)

## 7-1 Kaggle Kernels
In the end , I want to thank all the kernels I've used in this notebook
1. [basic-feature-benchmark](https://www.kaggle.com/inversion/basic-feature-benchmark)

Go to first step: [Course Home Page](https://www.kaggle.com/mjbahmani/10-steps-to-become-a-data-scientist)

Go to next step : [Titanic](https://www.kaggle.com/mjbahmani/a-comprehensive-ml-workflow-with-python)


# Not Completed yet!!!